In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patch
from PyQt5.QtWidgets import *
import PyQt5.QtCore
import PyQt5.QtGui
from bokeh.palettes import Category20_20
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg
import sys
import re

In [2]:
class CelestialBody:
    """
    Represents a generalized celestial body.
    
    Attributes
    ---
    name : str
        Unique name of celestial body.
    mass_kg : int
        Mass of celestial body [kg].
    radius_m : int
        Radius of celestial body [m].
    G : int
        Gravitational constant.

    """
    G = 6.673e-11

    def __init__(self, bodyName, bodyMass, bodyRadius):
        """
        Creates a celestial body with given parameters.
        
        Parameters
        ---
        bodyName : str
            Unique name of celestial body.
        bodyMass : int
            Mass of celestial body (>= 0) [kg].
        bodyRadius : int
            Radius of celestial body (>= 0) [m].

        """
        self.name = bodyName
        self.mass_kg = bodyMass
        self.radius_m = bodyRadius
        
    def calcGravForce(self, celestialBody, distance):
        """
        Returns the mutual gravitational force between the two celestial
        bodies in Newtons.
        
        Parameters
        ---
        celestialBody : CelestialBody
            Other celestial body to calculate mutual gravity between.
        distance : float
            Distance between the two celestial bodies.
        
        Returns
        ---
        float
            Mutual Gravitational Force in newtons
        
        """
        return(self.G*self.mass_kg*celestialBody.mass_kg/distance**2)
    
    def printAttr(self):
        """
        Print the current attributes of the celestial body to console.
        
        """
        print("Body Name: "+self.name)
        print("Body Mass: "+str(self.mass_kg)+" kg")
        print("Body Radius: "+str(self.radius_m)+" m")
    

In [3]:
class Sun(CelestialBody):
    """
    Specialized celestial body representing humanity's Sun.
    
    Attrbutes
    ---
    x : float
        x coordinate of Sun for plotting orbits relative to.
    y : float
        y coordinate of Sun for plotting orbits relative to.
    
    """
    x = 0.
    y = 0.
    
    def __init__(self):
        """
        Creates a celestial body of the Sun with additional plotting
        coordinates to use as origin.
        
        """
        super().__init__('Sun', 1.988e30, 6.957e8)


In [4]:
class SolarOrbit:
    """
    Represents a celestial body's orbit around the Sun.
    
    Attributes
    ---
    orbitalEccentricity : float
        Eccentricity of orbit [No units].
    semiMajorAxis : float
        Semi major axis of orbit [Astronomical Units].
    planet : CelestialBody
        Planet orbitting sun.
    theta : float
        Angle of planet in polar coordinates [Radians].
    radius : float
        Radius of planet in polar coordinates [Astronomical Units].
    sun : CelestialBody
        CelestialBody with information on the Sun.
    periodSeconds : float
        Period of orbit [Seconds].
    deltaThetaPerEarthDay : float
        Amount theta changes per earth day [Radians].

    """
    
    def __init__(self, planetaryBody, semiMajorAxis, orbitalEccentricity,
                 theta=0.):
        """
        Creates a SolarOrbit object representing orbit of given planetary
        body.
        
        Parameters
        ---
        planetaryBody : CelestialBody
            CelestialBody object containing planet's information.
        semiMajorAxis : float
            Semi major axis of planet's orbit (> 0) [au].
        orbitalEccentricity : float
            Eccentricity of the planet's orbit (0 < orbitalEccentricity < 1).
        theta : float, optional
            Initial theta coordinate for planet in radians, default is 0
            radians.

        """
        
        self.semiMajorAxis = semiMajorAxis
        self.orbitalEccentricity = orbitalEccentricity
        self.planet = planetaryBody
        self.theta = theta
        self.radius = self._calcEllipticalRadius(self.theta)
        self.sun = Sun()
        self.periodSeconds = self._calcPeriodSeconds(
            self.semiMajorAxis, self.planet, self.sun)
        self.deltaThetaPerEarthDay = self._calcThetaMotionPerEarthDay(
            self.periodSeconds)
        self._updateCartesianCoords()

    def incrementTheta(self, thetaIncrement):
        """
        Increments theta of SolarOrbit and updates the radius and cartesian
        scoordinates accordingly.
        
        Parameters
        ---
        thetaIncrement : float
            Number of radians to increment theta by.

        """
        self.theta += thetaIncrement
        self.radius = self._calcEllipticalRadius(self.theta)
        self._updateCartesianCoords()
    
    def incrementThetaNEarthDays(self, nDays):
        """
        Increments planet's position by theta equivalent to the change made
        in n earth days.
        
        Parameters
        ---
        nDays : float
            Decimal number of earth days to implement theta by.
        
        """
        self.incrementTheta(nDays*self.deltaThetaPerEarthDay)
    
    def getCartesianCoordinates(self):
        """
        Returns cartesian coordinate's of planet relative to Sun's.
        
        Returns
        ---
        
        tuple (float, float)
            x then y coordinate relative to Sun coordinate

        """
        return((self.x-self.sun.__class__.x, self.y-self.sun.__class__.y))

    def _calcEllipticalRadius(self, theta):
        return(self.semiMajorAxis*(1-self.orbitalEccentricity**2)/
               (1+self.orbitalEccentricity*np.cos(theta)))
    
    def _calcPeriodSeconds(self, semiMajorAxis, planet, sun):
        return(np.sqrt(4*np.pi**2*(1.496e11*semiMajorAxis)**3/
                       (planet.G*(planet.mass_kg+sun.mass_kg))))
    
    def _calcThetaMotionPerEarthDay(self, period):
        periodInDays = period/86400
        return(2*np.pi/periodInDays)

    def _updateCartesianCoords(self):
        self.x = self.radius*np.cos(self.theta)
        self.y = self.radius*np.sin(self.theta)


In [5]:
import unittest as ut

class TestCelestialBody(ut.TestCase):
    earth = None
    jupiter = None
    
    def setUp(self):
        self.earth = CelestialBody('Earth', 5.972e24, 6.371e6)
        self.jupiter = CelestialBody('Jupiter', 1.898e27, 6.991e7)
    
    def testConstructor(self):
        earth = CelestialBody('Earth', 5.972e24, 6.371e6)
        self.assertEqual(earth.name, 'Earth')
        self.assertEqual(earth.mass_kg, 5.972e24)
        self.assertEqual(earth.radius_m, 6.371e6)
        jupiter = CelestialBody('Jupiter', 1.898e27, 6.991e7)
        self.assertEqual(jupiter.name, 'Jupiter')
        self.assertEqual(jupiter.mass_kg, 1.898e27)
        self.assertEqual(jupiter.radius_m, 6.991e7)
    
    def testCalcGravForce(self):
        self.assertTrue(7.563e23 <= self.earth.calcGravForce(self.jupiter, 1e9) <= 7.564e23)
    
class TestSolarOrbit(ut.TestCase):
    earth = CelestialBody('Earth', 5.972e24, 6.371e6)
    
    def testConstructor(self):
        earthOrbit = SolarOrbit(self.earth, 1, 0.0167)
        sun = earthOrbit.sun
        self.assertEqual(sun.name, 'Sun')
        self.assertEqual(sun.mass_kg, 1.988e30)
        self.assertEqual(sun.radius_m, 6.957e8)
        self.assertEqual(sun.__class__.x, Sun().__class__.x)
        self.assertEqual(sun.__class__.y, Sun().__class__.y)
        self.assertEqual(earthOrbit.orbitalEccentricity, 0.0167)
        self.assertEqual(earthOrbit.semiMajorAxis, 1)
        self.assertTrue(31565149. <= earthOrbit.periodSeconds <= 31565150.)
        self.assertTrue(0.01719 <= earthOrbit.deltaThetaPerEarthDay <= 0.01720)
        earth = earthOrbit.planet
        self.assertEqual(earth.radius_m, self.earth.radius_m)
        self.assertEqual(earth.mass_kg, self.earth.mass_kg)
        self.assertEqual(earthOrbit.theta, 0.)
        self.assertTrue(.98330 <= earthOrbit.radius <= .98331)
        x, y = earthOrbit.getCartesianCoordinates()
        self.assertTrue(.98330 <= x <= .98331)
        self.assertTrue(0. <= y <= 0.0001)
    
    def testIncrementTheta(self):
        earthOrbit = SolarOrbit(self.earth, 1, 0.0167)
        earthOrbit.incrementTheta(0.01745)
        self.assertTrue(0.01745 <= earthOrbit.theta <= 0.01746)
        self.assertTrue(0.9833 <= earthOrbit.radius <= 0.9834)
    
    def testIncrementNEarthDays(self):
        earthOrbit = SolarOrbit(self.earth, 1, 0.0167)
        earthOrbit.incrementThetaNEarthDays(1)
        self.assertTrue(0.01719 <= earthOrbit.theta <= 0.01720)
        self.assertTrue(0.9833 <= earthOrbit.radius <= 0.9834)
        earthOrbit.incrementThetaNEarthDays(7)
        self.assertTrue(0.1375 <= earthOrbit.theta <= 0.1376)
        self.assertTrue(0.9834 <= earthOrbit.radius <= 0.9835)

if __name__ == '__main__':
    ut.main(argv=['first-arg-is-ignored'], exit=False)

.....
----------------------------------------------------------------------
Ran 5 tests in 0.008s

OK


In [6]:
class SolarOrbitSimulation(QWidget):
    orbits = []
    planetPatches = []
    fps = 20.

    def __init__(self):
        super().__init__()
        self.setWindowTitle("Solar Orbital Simulation")
        layout = QGridLayout()
        layout.setSpacing(0)

        self.spaceFigure, self.spaceAx = self._returnInitialSpacePlot()
        self._drawSunOnPlot(self.spaceAx)
        self.spaceCanvas = FigureCanvasQTAgg(self.spaceFigure)
        forceCanvas = FigureCanvasQTAgg(self._returnForcePlot())
        forceCanvas2 = FigureCanvasQTAgg(self._returnForcePlot())
        layout.addWidget(self.spaceCanvas, 0, 0, 8, 1)
        layout.addWidget(forceCanvas, 0, 1, 2, 1)
        layout.addWidget(forceCanvas2, 2, 1, 2, 1)
        
        vbox = QVBoxLayout()
        
        self.timer = PyQt5.QtCore.QTimer(self)
        self.timer.timeout.connect(self._tickOrbits)
        
        startMotionButton = QPushButton('Start Planetary Motion')
        startMotionButton.clicked.connect(lambda: self.timer.start(1000./self.fps))
        vbox.addWidget(startMotionButton)
        
        stopMotionButton = QPushButton('Stop Planetary Motion')
        stopMotionButton.clicked.connect(lambda: self.timer.stop())
        vbox.addWidget(stopMotionButton)
        
        addNewOrbitButton = QPushButton('Add New Planet')
        addNewOrbitButton.clicked.connect(lambda: self._addNewPlanetaryOrbitDialog())
        vbox.addWidget(addNewOrbitButton)
        
        editRemoveOrbitButton = QPushButton('Remove/edit Planet')
        editRemoveOrbitButton.clicked.connect(lambda: self._planetaryOrbitListDialog())
        vbox.addWidget(editRemoveOrbitButton)

        addSolarSystemButton = QPushButton('Add Solar System')
        addSolarSystemButton.clicked.connect(lambda: self._addSolarSystem())
        vbox.addWidget(addSolarSystemButton)
        
        addTerrestialsButton = QPushButton('Add Terrestial Planets')
        addTerrestialsButton.clicked.connect(lambda: self._addTerrestialPlanets())
        vbox.addWidget(addTerrestialsButton)
        
        addGasGiantsButton = QPushButton('Add Gas Giants')
        addGasGiantsButton.clicked.connect(lambda: self._addGasGiantPlanets())
        vbox.addWidget(addGasGiantsButton)
        
        layout.addLayout(vbox, 4, 1)
        
        self.setLayout(layout)
        self.show()
        self.setWindowState(self.windowState() & ~PyQt5.QtCore.Qt.WindowMinimized | PyQt5.QtCore.Qt.WindowActive)
        self.activateWindow()
        self.raise_()
        self._rescaleSpaceView()

    def _returnInitialSpacePlot(self):
        fig, ax = plt.subplots(figsize=(10, 10))
        ax.set_facecolor('black')
        ax.tick_params(axis='both', which='both', left=False, labelleft=False,
                       bottom=False, labelbottom=False)
        fig.tight_layout()
        return(fig, ax)
    
    def _drawSunOnPlot(self, ax):
        sun = Sun()
        self.sunPatch = patch.Circle((sun.x, sun.y,), radius=0.1, color='yellow')
        ax.add_patch(self.sunPatch)
    
    def _drawNewPlanetSpacePlot(self, solarOrbit, planetColor):
        x, y = solarOrbit.getCartesianCoordinates()
        planetPatch = patch.Circle((x, y), radius=0.05, color=planetColor)
        self.spaceAx.add_patch(planetPatch)
        self.planetPatches.append(planetPatch)
        self._rescaleSpaceView()
    
    def _tickOrbits(self):
        for solarOrbit, planetPatch in zip(self.orbits, self.planetPatches):
            solarOrbit.incrementThetaNEarthDays(4./self.fps)
            planetPatch.center = solarOrbit.getCartesianCoordinates()
        self._redrawSpacePlot()
    
    def _rescaleSpaceView(self):
        maxDist = 1  # Minimum size is 2 au
        for orbit in self.orbits:
            a = orbit.semiMajorAxis
            e = orbit.orbitalEccentricity
            maxTemp = a*(1+e)
            if maxDist < maxTemp:
                maxDist = maxTemp
        self.spaceAx.set_xlim(-1.05*maxDist, 1.05*maxDist)
        self.spaceAx.set_ylim(-1.05*maxDist, 1.05*maxDist)
        self._redrawSpacePlot()
    
    def _redrawSpacePlot(self):
        self.spaceCanvas.draw()

    def _returnForcePlot(self):
        fig, ax = plt.subplots(figsize=(5, 2.5))
        ax.plot(np.linspace(0, 2*np.pi, 360),
                np.sin(np.linspace(0, 2*np.pi, 360)),
                'b--')
        ax.set_title('GravitationalForce('+r'$\theta$'+')')
        ax.set_xlim(0, 2*np.pi)
        fig.tight_layout()
        return(fig)
    
    def _addNewPlanetaryOrbitDialog(self):
        """
        Opens dialog to add a new planetary orbit to simulation.

        """
        self._PlanetaryOrbitDialog(title="Define New Planetary Orbit")
    
    def _removeEditPlanetaryOrbitDialog(self, orbit, planetColor):
        """
        Opens dialog to remove or edit the planet orbit.
        
        Parameters
        ---
        orbit : SolarOrbit
            Solar orbit to edit or remove.
    
        planetColor : tuple
            Color of planet in orbit.
        
        """
        planet = orbit.planet
        color = '#{0:02x}{1:02x}{2:02x}'.format(
            int(255*planetColor[0]), int(255*planetColor[1]),
            int(255*planetColor[2]))
        planetMassCoeff, planetMassExp = self._getCoeffExpScieNot(planet.mass_kg)
        planetRadCoeff, planetRadExp = self._getCoeffExpScieNot(planet.radius_m)
        self._PlanetaryOrbitDialog(
            planet.name, str(planetMassCoeff), str(planetMassExp), str(planetRadCoeff), str(planetRadExp),
            str(orbit.semiMajorAxis), str(orbit.orbitalEccentricity), orbit.theta,
            title="Remove/Edit Planetary Orbit", editing=True, iniColor=color)

    def _getCoeffExpScieNot(self, number):
        """
        Returns coefficient and exponent to scientific notation form of number.
        
        Parameters
        ---
        
        number : float
            Number to get coefficient and exponent of.
        
        Returns
        ---
        
        coeff : float
            Coefficient of number.
        
        exponent : int
            Exponent of number.
        
        """
        exponent = 0
        coeff = float(number)
        if coeff >= 1:
            while coeff > 10.:
                exponent += 1
                coeff /= 10.
        else:
            while coeff < 1.:
                exponent -= 1
                coeff *= 10.
        return(coeff, exponent)
    
    def _PlanetaryOrbitDialog(self, iniPlanetName='', iniPlanetMassCoefficient='', iniPlanetMassExponent='',
                              iniPlanetRadiusCoefficient='', iniPlanetRadiusExponent='',
                              iniOrbitSMA='', iniOrbitEcc='', iniTheta=0.,
                              title='', editing=False, iniColor='#183232'):
        """
        Sets up window, line edits, and labels for planetary orbit dialog.
            
        """
        dialog = QDialog(self)
        dialog.setWindowTitle(title)
        dialog.resize(400, 100)

        layout = QGridLayout()
        layout.setSpacing(10)
    
        label1 = QLabel('Planet Name')
        label1.setFont(PyQt5.QtGui.QFont('default', 10))
        planetName = QLineEdit(iniPlanetName)
        if editing:
            planetName.setDisabled(True)
        layout.addWidget(label1, 0, 0, 1, 8)
        layout.addWidget(planetName, 1, 0, 1, 8)

        label2 = QLabel('Planet Mass [kg]')
        label2.setFont(PyQt5.QtGui.QFont('default', 10))
        layout.addWidget(label2, 2, 0, 1, 4)
        planetMassCoefficient = QLineEdit(iniPlanetMassCoefficient)
        label21 = QLabel('10^')
        label21.setFont(PyQt5.QtGui.QFont('default', 10))
        planetMassExponent = QLineEdit(iniPlanetMassExponent)
        layout.addWidget(planetMassCoefficient, 3, 0, 1, 2)
        layout.addWidget(label21, 3, 2)
        layout.addWidget(planetMassExponent, 3, 3)
        
        label3 = QLabel('Planet Radius [m]')
        label3.setFont(PyQt5.QtGui.QFont('default', 10))
        layout.addWidget(label3, 2, 4, 1, 4)
        planetRadiusCoefficient = QLineEdit(iniPlanetRadiusCoefficient)
        label31 = QLabel('10^')
        label31.setFont(PyQt5.QtGui.QFont('default', 10))
        planetRadiusExponent = QLineEdit(iniPlanetRadiusExponent)
        layout.addWidget(planetRadiusCoefficient, 3, 4, 1, 2)
        layout.addWidget(label31, 3, 6)
        layout.addWidget(planetRadiusExponent, 3, 7)
        
        label4 = QLabel('Semi-major Axis [au]')
        label4.setFont(PyQt5.QtGui.QFont('default', 10))
        layout.addWidget(label4, 4, 0, 1, 4)
        orbitSMA = QLineEdit(iniOrbitSMA)
        layout.addWidget(orbitSMA, 5, 0, 1, 4)
        
        label5 = QLabel('Orbital Eccentricity')
        label5.setFont(PyQt5.QtGui.QFont('default', 10))
        layout.addWidget(label5, 4, 4, 1, 4)
        orbitEccentricity = QLineEdit(iniOrbitEcc)
        layout.addWidget(orbitEccentricity, 5, 4, 1, 4)
        
        colorDialog = QColorDialog()
        colorDialog.setOption(QColorDialog.NoButtons)
        colorDialog.setCurrentColor(PyQt5.QtGui.QColor(iniColor))
        layout.addWidget(colorDialog, 6, 0, 1, 8)
        
        if not editing:
            addButton = QPushButton('Add Planetary Orbit')
            addButton.setFont(PyQt5.QtGui.QFont('default', 10))
            addButton.clicked.connect(
                lambda: self._confirmValidNewOrbitDefinition(
                    planetName.text(), planetMassCoefficient.text(),
                    planetMassExponent.text(), planetRadiusCoefficient.text(),
                    planetRadiusExponent.text(), orbitSMA.text(),
                    orbitEccentricity.text(), dialog,
                    colorDialog.currentColor().name()))
            layout.addWidget(addButton, 7, 0, 1, 8)
        else:
            addEditsButton = QPushButton('Add Orbit Edits')
            addEditsButton.setFont(PyQt5.QtGui.QFont('default', 10))
            addEditsButton.clicked.connect(
                lambda: self._confirmEditedOrbitDefinition(
                    planetName.text(), planetMassCoefficient.text(),
                    planetMassExponent.text(), planetRadiusCoefficient.text(),
                    planetRadiusExponent.text(), orbitSMA.text(),
                    orbitEccentricity.text(), iniTheta, dialog,
                    colorDialog.currentColor().name()))
            layout.addWidget(addEditsButton, 7, 0, 1, 4)
            removeButton = QPushButton('Remove Orbit')
            removeButton.setFont(PyQt5.QtGui.QFont('default', 10))
            removeButton.clicked.connect(
                lambda: [self._removeOrbit(iniPlanetName),
                         dialog.close()]
            )
            layout.addWidget(removeButton, 7, 4, 1, 4)
            
        dialog.setLayout(layout)
        dialog.exec_()
    
    def _createOrbitEntryErrorMsg(self, planetName, planetMassCoefficient, planetMassExponent,
                                  planetRadiusCoefficient, planetRadiusExponent, orbitSMA,
                                  orbitalEccentricity, checkName=True):
        errorMsg = ''
        if checkName:
            if planetName == '' or self.orbitContainsPlanet(planetName):
                errorMsg += 'Planet must have name that is unique.\n'
        if re.fullmatch(r'((\d+\.?\d*)|(\.\d+))', planetMassCoefficient) is None:
            errorMsg += 'Planet mass coefficient must be valid float.\n'
        if not (planetMassExponent.isdigit() or
                (planetMassExponent[1:].isdigit() and
                 planetMassExponent[0] == '-')):
            errorMsg += 'Planet mass exponent must be valid int.\n'
        if re.fullmatch(r'((\d+\.?\d*)|(\.\d+))', planetRadiusCoefficient) is None:
            errorMsg += 'Planet radius coefficient must be valid float.\n'
        if not (planetRadiusExponent.isdigit() or
                (planetRadiusExponent[1:].isdigit() and
                 planetRadiusExponent[0] == '-')):
            errorMsg += 'Planet radius exponent must be valid int.\n'
        if re.fullmatch(r'((\d+\.?\d*)|(\.\d+))', orbitSMA) is None:
            errorMsg += 'Semi major axis must be valid float.\n'
        if ((re.fullmatch(r'((\d+\.?\d*)|(\.\d+))', orbitalEccentricity) is None) or
            not (0 <= float(orbitalEccentricity) < 1)):
            errorMsg += 'Orbital eccentricity must be float in [0, 1)'
        return(errorMsg)

    def _confirmValidNewOrbitDefinition(self, planetName, planetMassCoefficient, planetMassExponent,
                                        planetRadiusCoefficient, planetRadiusExponent, orbitSMA,
                                        orbitalEccentricity, dialog, color):
        errorMsg = self._createOrbitEntryErrorMsg(
            planetName, planetMassCoefficient, planetMassExponent,
            planetRadiusCoefficient, planetRadiusExponent, orbitSMA,
            orbitalEccentricity, checkName=True)

        if errorMsg == '':
            self._addOrbit(planetName,
                           float(planetMassCoefficient)*10**int(planetMassExponent),
                           float(planetRadiusCoefficient)*10**int(planetRadiusExponent),
                           float(orbitSMA),
                           float(orbitalEccentricity),
                           color)
            dialog.close()
        else:
            QMessageBox.question(dialog, 'Input Error', errorMsg, QMessageBox.Ok)
    
    def _confirmEditedOrbitDefinition(self, planetName, planetMassCoefficient, planetMassExponent,
                                      planetRadiusCoefficient, planetRadiusExponent, orbitSMA,
                                      orbitalEccentricity, theta, dialog, color):
        errorMsg = self._createOrbitEntryErrorMsg(
            planetName, planetMassCoefficient, planetMassExponent,
            planetRadiusCoefficient, planetRadiusExponent, orbitSMA,
            orbitalEccentricity, checkName=False)

        if errorMsg == '':
            self._removeOrbit(planetName)
            self._addOrbit(planetName,
                           float(planetMassCoefficient)*10**int(planetMassExponent),
                           float(planetRadiusCoefficient)*10**int(planetRadiusExponent),
                           float(orbitSMA),
                           float(orbitalEccentricity),
                           color,
                           theta)
            dialog.close()
        else:
            QMessageBox.question(dialog, 'Input Error', errorMsg, QMessageBox.Ok)
    
    def _planetaryOrbitListDialog(self):
        dialog = QDialog(self)
        dialog.setWindowTitle('Select Planetary Orbit To Edit')
        dialog.resize(400, 0)
        layout = QVBoxLayout()
        tempDict = {}
        for orbit, planetPatch in zip(self.orbits, self.planetPatches):
            button = QPushButton(orbit.planet.name)
            button.clicked.connect(
                lambda checked, arg=button.text() : [
                         dialog.close(),
                         self.timer.stop(),
                         self._removeEditPlanetaryOrbitDialog(
                             tempDict[arg][0], tempDict[arg][1])])
            layout.addWidget(button)
            tempDict.update([(orbit.planet.name, (orbit, planetPatch.get_facecolor()))])
        dialog.setLayout(layout)
        dialog.exec_()

    def _addOrbit(self, planetName, planetMass, planetRadius, semiMajorAxis,
                  orbitalEccentricity, color, theta=0.):
        """
        If orbits does not have planet, add planet to orbits and draw.

        """
        if not self.orbitContainsPlanet(planetName):
            planet = CelestialBody(planetName, planetMass, planetRadius)
            planetaryOrbit = SolarOrbit(planet, semiMajorAxis,
                                        orbitalEccentricity, theta)
            self.orbits.append(planetaryOrbit)
            self._drawNewPlanetSpacePlot(planetaryOrbit, color)
    
    def _removeOrbit(self, planetName):
        """
        Removes orbit with planet of planetName if is in orbits.
        
        Parameters
        ---
        planetName : str
            Name of planet to remove.
        
        """
        if self.orbitContainsPlanet(planetName):
            names = [o.planet.name for o in self.orbits]
            index = names.index(planetName)
            del self.orbits[index]
            self.planetPatches[index].remove()
            del self.planetPatches[index]
            self._rescaleSpaceView(),
            self._redrawSpacePlot()
    
    def _addSolarSystem(self):
        self._addTerrestialPlanets()
        self._addGasGiantPlanets()
    
    def _addTerrestialPlanets(self):
        self._addOrbit('Mercury', 3.301e23, 2.440e6, 0.3871, 0.2056, '#b5b5b3')
        self._addOrbit('Venus', 4.867e24, 6.052e6, 0.7233, 0.006777, '#ffce52')
        self._addOrbit('Earth', 5.972e24, 6.371e6, 1, 0.01671, '#72c5d4')
        self._addOrbit('Mars', 6.417e23, 3.390e6, 1.523, 0.09339, '#ff0000')
    
    def _addGasGiantPlanets(self):
        self._addOrbit('Jupiter', 1.898e27, 6.991e7, 5.203, 0.04839, '#fa461e')
        self._addOrbit('Saturn', 5.683e26, 5.823e7, 9.539, 0.05386, '#e8d046')
        self._addOrbit('Uranus', 8.681e25, 2.536e7, 19.19, 0.04726, '#91ffe4')
        self._addOrbit('Neptune', 1.024e26, 2.462e7, 30.10, 0.008590, '#2491ff')

    def orbitContainsPlanet(self, planetName):
        """
        Returns true if any orbits contain planet with planetName.
        
        Parameters
        ---
        planetName : str
            Planet name to check if in orbits.

        Returns
        ---
        bool
            If planet in orbits or not.
        
        """
        for orbit in self.orbits:
            if orbit.planet.name == planetName:
                return(True)
        return(False)
        
    def closeEvent(self, event):
        plt.close('all')
        self.close()

if __name__ == "__main__":
    app = QApplication([])
    solarOrbitSim = SolarOrbitSimulation()
    sys.exit(app.exec_())

SystemExit: 0

C:\Users\dosre\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2971: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
